In [1]:
#note datalink: https://www.kaggle.com/johnolafenwa/us-census-data

#version:
scikit-learn==0.19.0 #0.18 does not work for EvolutionaryAlgorithmSearchCV
EvolutionaryAlgorithmSearchCV is in sklearn-deap

In [2]:


import pandas as pd
import numpy as np
from plotnine import *
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn_pandas import DataFrameMapper
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

/home/wxk/anaconda2/envs/py3/lib/python3.6/site-packages/matplotlib/colors.py:680: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  not cbook.is_string_like(colors[0]):
/home/wxk/anaconda2/envs/py3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
project_name = "USIncome"
data_dir = "/home/wxk/Data/%s"%(project_name)

training_data = data_dir+'/adult-training.csv'
test_data = data_dir+'/adult-test.csv'


In [4]:


columns = ['Age','Workclass','fnlgwt','Education','EdNum','MaritalStatus',
           'Occupation','Relationship','Race','Sex','CapitalGain','CapitalLoss',
           'HoursPerWeek','Country','Income']

df_train_set = pd.read_csv(training_data, names=columns)
df_test_set = pd.read_csv(test_data, names=columns, skiprows=1)
df_train_set.drop('fnlgwt', axis=1, inplace=True)
df_test_set.drop('fnlgwt', axis=1, inplace=True)

In [5]:
#data clean
for i in df_train_set.columns:
    df_train_set[i].replace(' ?', 'Unknown', inplace=True)
    df_test_set[i].replace(' ?', 'Unknown', inplace=True)

for col in df_train_set.columns:
    if df_train_set[col].dtype != 'int64':
        df_train_set[col] = df_train_set[col].apply(lambda val: val.replace(" ", ""))
        df_train_set[col] = df_train_set[col].apply(lambda val: val.replace(".", ""))
        df_test_set[col] = df_test_set[col].apply(lambda val: val.replace(" ", ""))
        df_test_set[col] = df_test_set[col].apply(lambda val: val.replace(".", ""))

In [6]:
df_train_set.drop(["Country", "Education"], axis=1, inplace=True)
df_test_set.drop(["Country", "Education"], axis=1, inplace=True)

colnames = list(df_train_set.columns)
colnames.remove('Age')
colnames.remove('EdNum')
colnames = ['AgeGroup', 'Education'] + colnames

labels = ["{0}-{1}".format(i, i + 9) for i in range(0, 100, 10)]
df_train_set['AgeGroup'] = pd.cut(df_train_set.Age, range(0, 101, 10), right=False, labels=labels)
df_test_set['AgeGroup'] = pd.cut(df_test_set.Age, range(0, 101, 10), right=False, labels=labels)

labels = ["{0}-{1}".format(i, i + 4) for i in range(0, 20, 5)]
df_train_set['Education'] = pd.cut(df_train_set.EdNum, range(0, 21, 5), right=False, labels=labels)
df_test_set['Education'] = pd.cut(df_test_set.EdNum, range(0, 21, 5), right=False, labels=labels)

df_train_set = df_train_set[colnames]
df_test_set = df_test_set[colnames]

In [7]:
mapper = DataFrameMapper([
    ('AgeGroup', LabelEncoder()),
    ('Education', LabelEncoder()),
    ('Workclass', LabelEncoder()),
    ('MaritalStatus', LabelEncoder()),
    ('Occupation', LabelEncoder()),
    ('Relationship', LabelEncoder()),
    ('Race', LabelEncoder()),
    ('Sex', LabelEncoder()),
    ('Income', LabelEncoder())
], df_out=True, default=None)

cols = list(df_train_set.columns)
cols.remove("Income")
cols = cols[:-3] + ["Income"] + cols[-3:]

df_train = mapper.fit_transform(df_train_set.copy())
df_train.columns = cols

df_test = mapper.transform(df_test_set.copy())
df_test.columns = cols

cols.remove("Income")
x_train, y_train = df_train[cols].values, df_train["Income"].values
x_test, y_test = df_test[cols].values, df_test["Income"].values

In [8]:
from sklearn.ensemble import AdaBoostClassifier
aclf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5), n_estimators=100)
aclf.fit(x_train, y_train)
aclf.score(x_test, y_test)

0.85516860143725815

In [9]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV

parameters = {
     'base_estimator__max_features':(11, 9, 6),
     'base_estimator__max_depth':(1, 2, 4, 8),
     #'base_estimator__min_samples_split': (2, 4, 8),
    # 'base_estimator__min_samples_leaf': (16, 12, 8, 4),
    # 'n_estimators': (50, 100, 200, 500),
     'learning_rate': (1, 0.1, 0.01, 10)
}



?EvolutionaryAlgorithmSearchCV

In [10]:
clf2 = EvolutionaryAlgorithmSearchCV(estimator=aclf,
                                   params=parameters,
                                   scoring="accuracy",
                                   cv=5,
                                   verbose=1,
                                   population_size=200,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=10,
                                   generations_number=100,
                                   n_jobs=8)

In [11]:
clf2.fit(x_train, y_train)

Types [1, 1, 1] and maxint [2, 3, 3] detected
--- Evolve in 48 possible combinations ---
gen	nevals	avg     	min     	max    	std     
0  	200   	0.705619	0.200086	0.87095	0.242905
1  	133   	0.845074	0.231289	0.87095	0.107344
2  	118   	0.869713	0.837659	0.87095	0.00505475
3  	115   	0.867442	0.231289	0.87095	0.0451629 
4  	127   	0.870031	0.837659	0.871103	0.00464587
5  	118   	0.867141	0.231289	0.871103	0.0452001 
6  	137   	0.864148	0.226774	0.871103	0.0628895 
7  	119   	0.870421	0.84202 	0.871472	0.00244005
8  	128   	0.866997	0.307853	0.871472	0.0401297 
9  	103   	0.867262	0.239028	0.871472	0.0446504 
10 	117   	0.866601	0.224778	0.871472	0.0457361 
11 	127   	0.870271	0.840085	0.871472	0.00332722
12 	119   	0.867516	0.315777	0.871472	0.0393138 
13 	133   	0.866833	0.229231	0.871472	0.0453344 


Process ForkPoolWorker-3:
Process ForkPoolWorker-6:
Process ForkPoolWorker-8:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-5:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/wxk/anaconda2/envs/py3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/wxk/anaconda2/envs/py3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/wxk/anaconda2/envs/py3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wxk/anaconda2/envs/py3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/wxk/anaconda2/envs/py3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self

  File "/home/wxk/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py", line 437, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/wxk/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py", line 473, in _boost
    return self._boost_real(iboost, X, y, sample_weight, random_state)
  File "/home/wxk/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py", line 483, in _boost_real
    estimator.fit(X, y, sample_weight=sample_weight)
KeyboardInterrupt
  File "/home/wxk/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py", line 413, in fit
    return super(AdaBoostClassifier, self).fit(X, y, sample_weight)
  File "/home/wxk/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py", line 413, in fit
    return super(AdaBoostClassifier, self).fit(X, y, sample_weight)
  File "/home/wxk/anaconda2/envs/py3/lib/python

KeyboardInterrupt: 